In [2]:
#Loading required R libraries 
library(quantmod)
library(plyr)
library(dplyr)
library(data.table)
library(DMwR)
library(mice)
defaultW <- getOption("warn") 
options(warn = -1) 

In [4]:
#Load dataset using Quantmod
stocklist <- c("DJI","DPZ","GOOG","IBM","AMZN","AMD","GM","F","GE","BA")
getSymbols(stocklist, from='2020-03-01',to='2020-06-01')
# To assign it from yahoo instead of Google
#getSymbols(stocklist,src="yahoo", auto.assign=FALSE, from='2020-03-01',to='2020-06-01')
options("getSymbols.warning4.0"=FALSE)

pausing 1 second between requests for more than 5 symbols
pausing 1 second between requests for more than 5 symbols
pausing 1 second between requests for more than 5 symbols
pausing 1 second between requests for more than 5 symbols
pausing 1 second between requests for more than 5 symbols
pausing 1 second between requests for more than 5 symbols


[1] "DJI"  "DPZ"  "GOOG" "IBM"  "AMZN" "AMD"  "GM"   "F"    "GE"   "BA"

In [5]:
#Merge the stock Details to form a portfolio
dji<-as.data.table(Ad(DJI))
dpz<-as.data.table(Ad(DPZ))
goog<-as.data.table(Ad(GOOG))
ibm<-as.data.table(Ad(IBM))
amzn<-as.data.table(Ad(AMZN))
amd<-as.data.table(Ad(AMD))
gm<-as.data.table(Ad(GM))
f<-as.data.table(Ad(F))
ge<-as.data.table(Ad(GE))
ba<-as.data.table(Ad(BA))
portfolio<-join_all(list(dji,dpz,goog,ibm,amzn,amd,gm,f,ge,ba), by='index', type='left')
colnames(portfolio)<-c("Date","DJI","DPZ","GOOG","IBM","AMZN","AMD","GM","F","GE","BA")
portfolio<-na.omit(portfolio)
head(portfolio)
tail(portfolio)


Date,DJI,DPZ,GOOG,IBM,AMZN,AMD,GM,F,GE,BA
2020-03-02,26703.3,345.9987,1389.11,132.5227,1953.95,47.46,31.04133,7.20,11.19888,289.27
2020-03-03,25917.4,337.9839,1341.39,127.1942,1908.99,46.75,30.15217,6.97,10.86921,280.62
2020-03-04,27090.9,340.3933,1386.52,132.4438,1975.83,50.11,31.15000,7.08,10.93914,283.12
2020-03-05,26121.3,331.5721,1319.04,127.8356,1924.03,48.11,30.10000,6.74,10.07000,260.37
2020-03-06,25864.8,334.9373,1298.41,126.0397,1901.09,48.59,28.69000,6.49,9.40000,262.33
2020-03-09,23851.0,335.1962,1215.56,116.2509,1800.61,43.27,24.69000,5.90,8.21000,227.17


Date,DJI,DPZ,GOOG,IBM,AMZN,AMD,GM,F,GE,BA
2020-05-21,24474.1,374.4754,1402.80,119.12,2446.74,54.65,25.80,5.63,6.48,139.00
2020-05-22,24465.2,371.3418,1410.42,118.39,2436.88,55.17,25.98,5.65,6.41,137.53
2020-05-26,24995.1,359.4261,1417.02,121.76,2421.86,53.19,27.40,5.84,6.80,144.73
2020-05-27,25548.3,373.2080,1417.84,125.54,2410.39,52.74,27.79,6.03,7.29,149.52
2020-05-28,25400.6,364.8051,1416.73,124.53,2401.10,51.74,26.68,5.85,6.78,149.82
2020-05-29,25383.1,385.0538,1428.92,124.90,2442.37,53.80,25.88,5.71,6.57,145.85


### Generate Missingness

In [6]:
df <- subset(portfolio, select = -Date )
#To set Percentage Missingness for Patterns
myfreq <- c(0.05,0.05, 0.075, 0.075, 0.1,0.1, 0.125,0.125,0.15,0.15)
amputedf<-ampute(df,prop=0.5,freq=myfreq,mech="MAR")


In [7]:
#check Pattern
amputedf$pattern

DJI,DPZ,GOOG,IBM,AMZN,AMD,GM,F,GE,BA
0,1,1,1,1,1,1,1,1,1
1,0,1,1,1,1,1,1,1,1
1,1,0,1,1,1,1,1,1,1
1,1,1,0,1,1,1,1,1,1
1,1,1,1,0,1,1,1,1,1
1,1,1,1,1,0,1,1,1,1
1,1,1,1,1,1,0,1,1,1
1,1,1,1,1,1,1,0,1,1
1,1,1,1,1,1,1,1,0,1
1,1,1,1,1,1,1,1,1,0


In [8]:
#Check Weight
amputedf$weights

DJI,DPZ,GOOG,IBM,AMZN,AMD,GM,F,GE,BA
0,1,1,1,1,1,1,1,1,1
1,0,1,1,1,1,1,1,1,1
1,1,0,1,1,1,1,1,1,1
1,1,1,0,1,1,1,1,1,1
1,1,1,1,0,1,1,1,1,1
1,1,1,1,1,0,1,1,1,1
1,1,1,1,1,1,0,1,1,1
1,1,1,1,1,1,1,0,1,1
1,1,1,1,1,1,1,1,0,1
1,1,1,1,1,1,1,1,1,0


In [9]:
#Check Frequency
amputedf$freq

[1] 0.050 0.050 0.075 0.075 0.100 0.100 0.125 0.125 0.150 0.150

In [10]:
#Display Amputed Dataset
head(amputedf$amp)

DJI,DPZ,GOOG,IBM,AMZN,AMD,GM,F,GE,BA
26703.3,345.9987,1389.11,132.5227,1953.95,47.46,31.04133,7.20,11.19888,NA
25917.4,337.9839,1341.39,NA,1908.99,46.75,30.15217,6.97,10.86921,280.62
27090.9,340.3933,1386.52,132.4438,1975.83,50.11,31.15000,7.08,10.93914,283.12
26121.3,331.5721,1319.04,127.8356,1924.03,48.11,NA,6.74,10.07000,260.37
25864.8,NA,1298.41,126.0397,1901.09,48.59,28.69000,6.49,9.40000,262.33
23851.0,335.1962,1215.56,116.2509,1800.61,43.27,24.69000,5.90,8.21000,227.17


### KNN to Predict the NA's with Method Median

In [11]:
knnImpute<-knnImputation(amputedf$amp, scale=T,k =3, meth = 'median')
#Add the date back again
knnImpute$Date<-portfolio$Date
#Re-Order Date as the first column
knnImpute<- knnImpute %>% select(Date,DJI,DPZ,GOOG,IBM,AMZN,AMD,GM,F,GE,BA)
head(knnImpute)

Date,DJI,DPZ,GOOG,IBM,AMZN,AMD,GM,F,GE,BA
2020-03-02,26703.3,345.9987,1389.11,132.5227,1953.95,47.46,31.04133,7.20,11.19888,231.01
2020-03-03,25917.4,337.9839,1341.39,123.1188,1908.99,46.75,30.15217,6.97,10.86921,280.62
2020-03-04,27090.9,340.3933,1386.52,132.4438,1975.83,50.11,31.15000,7.08,10.93914,283.12
2020-03-05,26121.3,331.5721,1319.04,127.8356,1924.03,48.11,27.12000,6.74,10.07000,260.37
2020-03-06,25864.8,340.3933,1298.41,126.0397,1901.09,48.59,28.69000,6.49,9.40000,262.33
2020-03-09,23851.0,335.1962,1215.56,116.2509,1800.61,43.27,24.69000,5.90,8.21000,227.17


In [12]:
#Write results to CSV
write.table(knnImpute, file = "Impute.csv")


In [13]:
#Read Covid Data
dfcovid <- read.csv(file = 'data/Covid19_6June/covid_19_data.csv', header = TRUE)
dfcovid <- subset(dfcovid, select =c(Date,Confirmed,Deaths,Recovered))
dfcovid <- dfcovid %>% group_by(Date) %>% summarise_all(sum)
dfcovid$Date<-as.Date(dfcovid$Date)
dfcovid <- subset(dfcovid, Date >= as.Date("2020-03-01") & Date<= as.Date("2020-06-01"))
head(dfcovid)

Date,Confirmed,Deaths,Recovered
2020-03-01,88371,2996,42716
2020-03-02,90309,3085,45602
2020-03-03,92844,3160,48229
2020-03-04,95124,3254,51171
2020-03-05,97886,3348,53797
2020-03-06,101800,3460,55866


### Merge Covid Dataset & Portfolio

In [14]:
#Merge Dataset of covid and Portfolio
dfcovidportfolio<-merge(knnImpute,dfcovid, by = c("Date"))
head(dfcovidportfolio)
tail(dfcovidportfolio)

Date,DJI,DPZ,GOOG,IBM,AMZN,AMD,GM,F,GE,BA,Confirmed,Deaths,Recovered
2020-03-02,26703.3,345.9987,1389.11,132.5227,1953.95,47.46,31.04133,7.20,11.19888,231.01,90309,3085,45602
2020-03-03,25917.4,337.9839,1341.39,123.1188,1908.99,46.75,30.15217,6.97,10.86921,280.62,92844,3160,48229
2020-03-04,27090.9,340.3933,1386.52,132.4438,1975.83,50.11,31.15000,7.08,10.93914,283.12,95124,3254,51171
2020-03-05,26121.3,331.5721,1319.04,127.8356,1924.03,48.11,27.12000,6.74,10.07000,260.37,97886,3348,53797
2020-03-06,25864.8,340.3933,1298.41,126.0397,1901.09,48.59,28.69000,6.49,9.40000,262.33,101800,3460,55866
2020-03-09,23851.0,335.1962,1215.56,116.2509,1800.61,43.27,24.69000,5.90,8.21000,227.17,113582,3996,62512


,Date,DJI,DPZ,GOOG,IBM,AMZN,AMD,GM,F,GE,BA,Confirmed,Deaths,Recovered
57,2020-05-21,24474.1,374.4754,1402.80,119.12,2446.74,54.59,25.80,5.63,6.48,139.00,5102424,332924,1948739
58,2020-05-22,24465.2,371.3418,1410.42,118.39,2436.88,55.17,24.81,5.65,6.41,137.53,5210817,338160,2056643
59,2020-05-26,24995.1,359.4261,1417.02,121.76,2421.86,53.19,27.40,5.84,6.80,144.73,5589626,350453,2286956
60,2020-05-27,25548.3,373.2080,1417.84,125.54,2410.39,52.74,27.79,6.03,7.29,144.73,5691790,355629,2350088
61,2020-05-28,25400.6,364.8051,1416.73,124.53,2401.10,51.74,26.68,5.85,6.78,149.82,5808946,360308,2415960
62,2020-05-29,25383.1,385.0538,1428.92,124.90,2421.86,53.80,25.88,5.71,6.57,145.85,5924275,364867,2493535
